In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
import os
py_file_location = "/content/drive/MyDrive/AI/WORK2/include"
sys.path.append(os.path.abspath(py_file_location))
import helpers
import IPython as IP
import pandas as pd
import numpy as np
#np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
import librosa
import pickle
import math
import librosa.display
#import warnings
#warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import scipy.io.wavfile

In [ ]:
# Load the metadata from the generated CSV
metadata = pd.read_csv('/content/drive/MyDrive/AI/WORK2/data/metadata/train.csv')
metadata.head()


In [ ]:
# Iterate through all audio files and extract MFCC
features = []
labels = []
frames_max = 0
counter = 0
total_samples = len(metadata)
n_mels = 30
duration = 1 #second
counter2 = 0
i = 0;
frame_length = 16896
hop_length = 8448
overlap_ratio = hop_length/frame_length
ERR = ['ERROR < 1']

In [ ]:
for index, row in metadata.iterrows():
    file_path = os.path.join('/content/drive/MyDrive/AI/WORK2/data/wav/train',str(row["slice_file_name"]))
    class_label = row["classID"]
    try:
        # Load audio file
        clip, sr = librosa.load(file_path, sr=16000, mono=True, dtype=np.float32)
        clip_length = len(clip)
        file_nums = math.floor(clip_length / hop_length) - 1
        for i in range(file_nums):
            y = clip[(i*hop_length):(i*hop_length+frame_length)]
            # Compute Mel
            mel = librosa.feature.melspectrogram(y=y, sr=16000, n_fft=1024, hop_length=512,
                                                 n_mels=n_mels, center=False)
            mel = mel / mel.max()
            # Chuyển đổi cường độ âm thanh thành biên độ log:
            mel_db = librosa.power_to_db(mel, top_db=80)
            # Normalize MFCC between -1 and 1
            normalized_mel = mel_db
            #normalized_mel = librosa.util.normalize(mel_db)   #my_mfcc = mfcc/(np.max(abs(mfcc)))
            mels = normalized_mel
            # Extract MFCCs (do not add padding)
            #mfccs = helpers.get_mfcc(file_path, 0, n_mfcc)

            # Save current frame count - số cột
            num_frames = mels.shape[1]
            # Add row (feature / label) - thêm vào list matran và label
            features.append(mels)
            labels.append(class_label)
            # Update frames maximum
            if num_frames > frames_max:
                frames_max = num_frames
                print(f"num_frames = {num_frames}")
                print(f"file name : {file_path}")
            # Notify update every N files
            if counter == 50:
                print("Status: {}/{}".format(index + 1, total_samples))
                counter = 0
            original_name = str(row[0])
            new_name = str(counter2)+'-'+original_name
            split_path = os.path.join('/content/drive/MyDrive/AI/WORK2/data/wav', 'pre_split', new_name)
            scipy.io.wavfile.write(split_path, sr, y)

            counter += 1
            counter2+= 1
            if  math.isnan(mel_db[29,20]):
                print(file_path)
    except Exception as e:
           print("Error parsing wavefile: ", e)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(f"num_frames = {labels}")
print(f"frame_max = {frames_max}")
print("Finished: {}/{}".format(index, total_samples))
# Add padding to features with less than frames than frames_max - Thêm phần đệm vào các đối tượng địa lý có ít khung hơn khung_max
padded_features = helpers.add_padding(features, frames_max)

In [ ]:
# Verify shapes
print("Raw features length: {}".format(len(features)))
print("Padded features length: {}".format(len(padded_features)))
print("Feature labels length: {}".format(len(labels)))
# Convert features (X) and labels (y) to Numpy arrays
X = np.array(padded_features)
y = np.array(labels)

In [ ]:
# Optionally save the features to disk
np.save("/content/drive/MyDrive/AI/WORK2/data/X-mel", X)
np.save("/content/drive/MyDrive/AI/WORK2/data/y-mel", y)
with open('/content/drive/MyDrive/AI/WORK2/data/parameter-mel.pickle', 'wb') as handle:
    pickle.dump([n_mels, frames_max], handle, protocol=pickle.HIGHEST_PROTOCOL)

print("End program")